# Data: Deriving Quantities

*Purpose*: Often our data will not tell us *directly* what we want to know; in
these cases we need to *derive* new quantities from our data. In this exercise,
we'll work with `tf_mutate()` to create new columns by operating on existing
variables, and use `tf_group_by()` with `tf_summarize()` to compute aggregate
statistics (summaries!) of our data.

Aside: The data-summary verbs in grama are heavily inspired by the [dplyr](https://dplyr.tidyverse.org/) package in the R programming langauge. 

## Setup

In [ ]:
import grama as gr
import pandas as pd
DF = gr.Intention()
%matplotlib inline

We'll be using the `diamonds` as seen in `e-vis00-basics` earlier. 

In [ ]:
from grama.data import df_diamonds
df_diamonds.head(10)

## Summarizing Data Frames

The `tf_summarize()` function in [grama](https://py-grama.readthedocs.io/en/latest/source/grama.dfply.html?highlight=tf_summarize#module-grama.dfply.summarize) is used to compute new variables and summary statistics. With `tf_summarize()` you can pass a data frame with arguments that assign the summary statistic's value and column name. Each argument should generate a single summary statistic for the output dataframe. 

| Type | Functions |
| ---- | --------- |
| Location | `gr.mean(x), gr.median(x), gr.mean_lo, gr.mean_up, gr.min(x), gr.max(x)` |
| Spread | `gr.sd(x), gr.var(x), gr.IQR(x)` |
| Position | `gr.first(x), gr.nth(x, n), gr.last(x)` |
| Counts | `gr.n_distinct(x), gr.n()` |
| Logical | `gr.sum(x != 0), gr.mean(y == 0)` |

### __q1__ Use `tf_summarize()` with a logical function

Using `summarize()` and a logical summary function, determine the number of rows with `Ideal` type of `cut`. Save this value to a column called `n_ideal`.

In [ ]:
# TASK: Determine the number of obsevations with an 'Ideal' diamond 'cut.' Assign this value as 'n_ideal'

df_q1 = (
    df_diamonds
    >> gr.tf_summarize(
# solution-start
        n_ideal=gr.sum(DF.cut == "Ideal")
# solution-end
    )
)

assert \
    df_q1["n_ideal"][0]/23==937,\
    "Sum does not match expected value"

df_q1

## Grouping multiple variables

The `tf_group_by()` takes a data frame and then the names of one or more columns in the data frame. It returns a copy of the data frame that has been “grouped” into sets of rows that share identical combinations of values in the specified columns. When passing a grouped data frame into `tf_summarize()` it will calculate your summary statistics ina group-wise manner. 

**DRAFT NOTE:** Copied directly from the RStudio article

### __q2__ How does `tf_group_by()` change the output to `tf_summarize()`?

The `tf_group_by()` modifies how other data-management verbs function. Uncomment the `tf_group_by()` below, and describe how the result changes.

In [ ]:
#TASK: Uncomment the tf_group_by() below, and describe how the result changes

df_q2 = (
    df_diamonds
#    >> gr.tf_group_by(DF["color"],DF["clarity"])
    >> gr.tf_summarize(diamonds_mean=gr.mean(DF["price"]))
)

df_q2

*Observations*
<!-- task-begin -->
- Write your observations here!
<!-- task-end -->
<!-- solution-begin -->
- The commented version computes a summary over the entire dataframe
- The uncommented version computes summaries over groups of `color` and `clarity`
<!-- solution-end -->


## Vectorized Functions

| Type | Functions |
| ---- | --------- |
| Arithmetic ops. | `+, -, *, /, ^` |
| Modular arith. | `//, %` |
| Logical comp. | `<, <=, >, >=, !=, ==` |
| Logarithms | `gr.log(x)` |
| Offsets | `gr.lead(x), gr.lag(x)` |
| Cumulants | `gr.cumsum(x), gr.cumprod(x), gr.cummin(x), gr.cummax(x), gr.cummean(x)` |
| Ranking | `gr.min_rank(x), gr.row_number(x), gr.dense_rank(x), gr.percent_rank(x)` |


### __q3__ Comment on why the difference is so large.

The `depth` variable is supposedly computed via `depth_computed = 100 * 2 * DF["z"] /
(DF["x"] + DF["y"])`. Compute `diff = DF["depth"] - DF["depth_computed"]`: This is a measure of
discrepancy between the given and computed depth. Additionally, compute the
*coefficient of variation* `cov = sd / mean` for both `depth` and `diff`:
This is a dimensionless measure of variation in a variable. Assign the resulting
tibble to `df_q3`, and assign the appropriate values to `cov_depth` and
`cov_diff`. Comment on the relative values of `cov_depth` and `cov_diff`; why is
`cov_diff` so large?

*Note*: Confusingly, the documentation for `diamonds` leaves out the factor of `100` in the computation of `depth`. Additionally, by default the `pandas` library will exclude values equal to `NaN` in functions like `gr.mean()` or `gr.sd()`

In [ ]:
df_q3 = (
    df_diamonds 
# solution-begin    
    >> gr.tf_mutate(depth_computed=100 * 2 * DF["z"] / (DF["x"] + DF["y"]))
    >> gr.tf_mutate(diff = DF["depth"]- DF["depth_computed"])
    >> gr.tf_mutate(
        depth_mean=gr.mean(DF["depth"]),
        depth_sd=gr.sd(DF["depth"]),
        diff_mean=gr.mean(DF["diff"]),
        diff_sd=gr.sd(DF["diff"]),
        diff_median = gr.median(DF["diff"])
    )
    >> gr.tf_mutate(
        cov_depth=DF["depth_sd"] / DF["depth_mean"],
        cov_diff=DF["diff_sd"] / DF["diff_mean"],
        c_diff=gr.IQR(DF["diff"]/DF["diff_median"])
    )
    >> gr.tf_select(DF["depth_mean"],DF["depth_sd"],DF["cov_depth"],DF["diff_mean"],DF["diff_sd"],DF["cov_diff"],DF["c_diff"])
# solution-end
)

assert \
    df_q3["cov_depth"][0] - 0.02320057 < 1e-3 and\
    df_q3["cov_diff"][0]- 497.5585 < 1e-3,\
    "Double check your calculations for depth and diff"

df_q3.head(1) 

#DRAFT NOTE: This is unelegant way to get the correct answer. Unclear how to pass forward variables in a summarize call or reorganize order with a select call

**Observation**

<!-- task-begin -->
- Comment on the relative values of `cov_depth` and `cov_diff`.
- Why is `cov_diff` so large?
<!-- task-end -->
<!-- solution-begin -->
- `cov_depth` is tiny; there's not much variation in the depth, compared to its scale.
- `cov_diff` is enormous! This is because the mean difference between `depth` and `depth_computed` is small, which causes the `cov` to blow up.
<!-- solution-end -->

### __q4__ Compute and observe

Compute the `price_mean = mean(price)`, `price_sd = sd(price)`, and `price_cov =
price_sd / price_mean` for each `cut` of diamond. What observations can you make
about the various cuts? Do those observations match your expectations?

In [ ]:
# TASK: Compute the `price_mean = gr.mean(DF["price"])`, `price_sd = gr.sd(DF["price"])`, and `price_cov = price_sd / price_mean` for each `cut` of diamond. 

df_q4=(
    df_diamonds
# solution-begin    
    >> gr.tf_group_by(DF["cut"])
    >> gr.tf_summarize(
        price_mean=gr.mean(DF["price"]),
        price_sd=gr.sd(DF["price"]),
    )
    >> gr.tf_mutate(price_cov=DF["price_sd"]/DF["price_mean"])
# solution-end
)

df_q4

In [ ]:
# DRAFT NOTE: Reading mutate_helpers.py I was expecting to be able to pass either a integer 3 after the DF.price_cov or pass the argument 'decimals = 3' per the np.round() documentation.
# Neither worked so I wasn't sure how to go about the assert for validating the provided answer. 

df_q4_check = (
    df_q4
        >> gr.tf_select(DF["cut"], DF["price_cov"]) 
        >> gr.tf_mutate(price_cov = gr.round(DF["price_cov"]))
        #>> gr.tf_mutate(price_cov = gr.round(DF["price_cov"], 3)) <- Expected  round call
)
df_q4_check

*Observations*
<!-- task-begin -->
- Write your observations here!
<!-- task-end -->
<!-- solution-begin -->
- I would expect the `Ideal` diamonds to have the highest price, but that's not the case!
- The `COV` for each cut is very large, on the order of 80 to 110 percent! To me, this implies that the other variables `clarity, carat, color` account for a large portion of the diamond price.
<!-- solution-end -->